# Strategy Comparison

Compare multiple strategies head-to-head with statistical significance testing and performance attribution.

In [1]:
from datetime import timedelta
from clyptq import CostModel, Constraints
from clyptq.analytics.performance.attribution import PerformanceAttributor
from clyptq.analytics.risk.monte_carlo import MonteCarloSimulator
from clyptq.data.loaders.ccxt import load_crypto_data
from clyptq.trading.engine import BacktestEngine
from clyptq.trading.execution import BacktestExecutor
from clyptq.trading.factors.library.momentum import MomentumFactor
from clyptq.trading.factors.library.volatility import VolatilityFactor
from clyptq.trading.factors.library.volume import VolumeFactor
from clyptq.trading.factors.library.mean_reversion import BollingerFactor, ZScoreFactor
from clyptq.trading.factors.library.liquidity import AmihudFactor
from clyptq.trading.factors.ops.cross_sectional import normalize
from clyptq.trading.portfolio.constructors import TopNConstructor, ScoreWeightedConstructor
from clyptq.trading.strategy.base import SimpleStrategy

## 1. Load Data

In [2]:
symbols = [
    "BTC/USDT", "ETH/USDT", "BNB/USDT", "SOL/USDT", "XRP/USDT",
    "ADA/USDT", "AVAX/USDT", "DOGE/USDT", "DOT/USDT", "MATIC/USDT",
    "LTC/USDT", "LINK/USDT", "UNI/USDT", "ATOM/USDT", "XLM/USDT",
    "ALGO/USDT", "FIL/USDT", "NEAR/USDT", "APT/USDT", "ARB/USDT"
]

store = load_crypto_data(symbols=symbols, exchange="binance", timeframe="1d", days=720)
date_range = store.get_date_range()
start = date_range.end - timedelta(days=365)
end = date_range.end

print(f"Testing {len(symbols)} symbols")
print(f"Period: {start.date()} to {end.date()}")

Testing 20 symbols
Period: 2025-01-04 to 2026-01-04


## 2. Define Strategy Variants

**4 Approaches**:
1. Momentum Only: Trend following
2. Mean Reversion: Contrarian
3. Multi-Factor: Combined signals
4. Long-Short: Market neutral

In [3]:
class MomentumOnlyStrategy(SimpleStrategy):
    """Pure momentum: buy strong trends, avoid volatility"""
    def __init__(self):
        super().__init__(
            factors_list=[MomentumFactor(lookback=30), VolatilityFactor(lookback=30)],
            constructor=TopNConstructor(top_n=5),
            constraints_obj=Constraints(max_position_size=0.20, max_gross_exposure=1.0, min_position_size=0.08, max_num_positions=8),
            schedule_str="weekly",
            warmup=35,
            name="Momentum"
        )

class MeanReversionStrategy(SimpleStrategy):
    """Buy oversold, sell overbought"""
    def __init__(self):
        super().__init__(
            factors_list=[BollingerFactor(lookback=20, num_std=2.0), ZScoreFactor(lookback=20), AmihudFactor(lookback=20)],
            constructor=TopNConstructor(top_n=5),
            constraints_obj=Constraints(max_position_size=0.25, max_gross_exposure=0.9, min_position_size=0.10, max_num_positions=5),
            schedule_str="weekly",
            warmup=25,
            name="MeanReversion"
        )

class MultiFactorStrategy(SimpleStrategy):
    """5 factors: momentum + mean reversion + risk + liquidity"""
    def __init__(self):
        super().__init__(
            factors_list=[
                MomentumFactor(lookback=30),
                BollingerFactor(lookback=20, num_std=2.0),
                VolatilityFactor(lookback=30),
                VolumeFactor(lookback=30),
                AmihudFactor(lookback=20),
            ],
            constructor=ScoreWeightedConstructor(use_long_short=False),
            constraints_obj=Constraints(max_position_size=0.20, max_gross_exposure=0.95, min_position_size=0.08, max_num_positions=7),
            schedule_str="weekly",
            warmup=35,
            name="MultiFactor"
        )

class LongShortStrategy(SimpleStrategy):
    """Long strong, short weak (market neutral)"""
    def __init__(self):
        super().__init__(
            factors_list=[
                MomentumFactor(lookback=30),
                BollingerFactor(lookback=20, num_std=2.0),
                VolatilityFactor(lookback=30),
                VolumeFactor(lookback=30),
                AmihudFactor(lookback=20),
            ],
            constructor=ScoreWeightedConstructor(use_long_short=True),
            constraints_obj=Constraints(max_position_size=0.20, max_gross_exposure=1.6, min_position_size=0.05, max_num_positions=10, allow_short=True),
            schedule_str="weekly",
            warmup=35,
            name="LongShort"
        )

## 3. Run Backtests

In [4]:
strategies = {
    "Momentum": MomentumOnlyStrategy(),
    "MeanReversion": MeanReversionStrategy(),
    "MultiFactor": MultiFactorStrategy(),
    "LongShort": LongShortStrategy(),
}

cost_model = CostModel(maker_fee=0.001, taker_fee=0.001, slippage_bps=5.0)
executor = BacktestExecutor(cost_model)

results = {}
for name, strategy in strategies.items():
    print(f"Running {name}...")
    engine = BacktestEngine(strategy=strategy, data_store=store, executor=executor, initial_capital=100000.0)
    results[name] = engine.run(start=start, end=end, verbose=False)

print("\nDone!")

Running Momentum...
Fill rejected: Insufficient cash for XRP/USDT: need 147.17, have 146.94
Fill rejected: Insufficient cash for BTC/USDT: need 103.02, have 102.83
Fill rejected: Insufficient cash for XRP/USDT: need 12606.41, have 12581.31
Running MeanReversion...
Fill rejected: Overselling ADA/USDT: trying to sell 31402.9894, only have 31394.6173
Fill rejected: Overselling DOGE/USDT: trying to sell 108783.0883, only have 108754.0865
Fill rejected: Overselling XLM/USDT: trying to sell 75499.1199, only have 75478.9916
Fill rejected: Insufficient cash for APT/USDT: need 18177.71, have 9804.58
Fill rejected: Insufficient cash for FIL/USDT: need 18177.71, have 9474.29
Fill rejected: Insufficient cash for MATIC/USDT: need 18177.71, have 9474.29
Fill rejected: Overselling ARB/USDT: trying to sell 62086.2490, only have 62071.2077
Fill rejected: Overselling ATOM/USDT: trying to sell 5488.9424, only have 5487.6126
Fill rejected: Overselling SOL/USDT: trying to sell 154.1343, only have 154.0970


## 4. Compare Performance

In [5]:
print("STRATEGY COMPARISON")
print("=" * 100)
print(f"{'Strategy':<20} {'Return':<12} {'Sharpe':<10} {'MaxDD':<10} {'Trades':<8} {'Win%':<8}")
print("=" * 100)

for name, result in results.items():
    m = result.metrics
    print(
        f"{name:<20} "
        f"{m.total_return:>10.2%}  "
        f"{m.sharpe_ratio:>8.3f}  "
        f"{m.max_drawdown:>8.2%}  "
        f"{len(result.trades):>6}  "
        f"{m.win_rate:>6.2%}"
    )

print("=" * 100)

best_name = max(results.items(), key=lambda x: x[1].metrics.sharpe_ratio)[0]
print(f"\nBest (by Sharpe): {best_name}")

STRATEGY COMPARISON
Strategy             Return       Sharpe     MaxDD      Trades   Win%    
Momentum                -37.37%    -1.562    43.21%      99  37.25%
MeanReversion           -27.85%    -0.627    54.27%     183  52.70%
MultiFactor              -6.89%    -2.387     8.49%       3   0.00%
LongShort               -38.02%    -1.255    42.06%     224  36.94%

Best (by Sharpe): MeanReversion


## 5. Detailed Analysis: Best Strategy

Deep dive into winner

In [6]:
best_result = results[best_name]
m = best_result.metrics

print(f"DETAILED ANALYSIS: {best_name}")
print("=" * 80)
print(f"\nPerformance:")
print(f"  Total Return:       {m.total_return:>10.2%}")
print(f"  Annualized Return:  {m.annualized_return:>10.2%}")
print(f"  Sharpe Ratio:       {m.sharpe_ratio:>10.3f}")
print(f"  Sortino Ratio:      {m.sortino_ratio:>10.3f}")
print(f"  Calmar Ratio:       {m.annualized_return / m.max_drawdown if m.max_drawdown > 1e-10 else 0:>10.3f}")
print(f"  Max Drawdown:       {m.max_drawdown:>10.2%}")
print(f"  Volatility:         {m.volatility:>10.2%}")

print(f"\nTrading:")
print(f"  Total Trades:       {m.num_trades:>10}")
print(f"  Win Rate:           {m.win_rate:>10.2%}")
print(f"  Profit Factor:      {m.profit_factor:>10.2f}")
print(f"  Avg Trade P&L:      ${m.avg_trade_pnl:>9.2f}")

DETAILED ANALYSIS: MeanReversion

Performance:
  Total Return:          -27.85%
  Annualized Return:     -46.67%
  Sharpe Ratio:           -0.627
  Sortino Ratio:          -1.651
  Calmar Ratio:           -0.860
  Max Drawdown:           54.27%
  Volatility:             74.48%

Trading:
  Total Trades:              183
  Win Rate:               52.70%
  Profit Factor:            1.57
  Avg Trade P&L:      $   329.77


## 6. Performance Attribution

Where did returns come from?

In [7]:
attributor = PerformanceAttributor()
attribution = attributor.analyze(best_result)

print("\nPERFORMANCE ATTRIBUTION")
print("=" * 80)

print("\nTop 10 Asset Contributors:")
sorted_assets = sorted(
    attribution.asset_attributions,
    key=lambda x: abs(x.weight_contribution),
    reverse=True
)[:10]

for asset_attr in sorted_assets:
    print(f"  {asset_attr.symbol:<12}: {asset_attr.weight_contribution:>8.2%}")

print(f"\nCost Analysis:")
print(f"  Transaction Costs:  {attribution.transaction_cost_drag:>8.2%}")
print(f"  Cash Drag:          {attribution.cash_drag:>8.2%}")


PERFORMANCE ATTRIBUTION

Top 10 Asset Contributors:
  UNI/USDT    :    0.00%
  XRP/USDT    :   -0.00%
  AVAX/USDT   :    0.00%
  DOGE/USDT   :   -0.00%
  BTC/USDT    :    0.00%
  ADA/USDT    :   -0.00%
  ATOM/USDT   :   -0.00%
  ALGO/USDT   :   -0.00%
  DOT/USDT    :   -0.00%
  XLM/USDT    :    0.00%

Cost Analysis:
  Transaction Costs:    -2.73%
  Cash Drag:             3.77%


## 7. Monte Carlo Risk Analysis

1000 simulations: bootstrap returns to estimate risk

In [8]:
mc = MonteCarloSimulator(num_simulations=1000, random_seed=42)
mc_result = mc.run(best_result)

print("\nMONTE CARLO SIMULATION (1000 runs)")
print("=" * 80)

print(f"\nReturn Distribution:")
print(f"  Mean:               {mc_result.mean_return:>10.2%}")
print(f"  Median:             {mc_result.median_return:>10.2%}")
print(f"  Std Dev:            {mc_result.std_return:>10.2%}")
print(f"  5th percentile:     {mc_result.ci_5_return:>10.2%}")
print(f"  95th percentile:    {mc_result.ci_95_return:>10.2%}")

print(f"\nRisk Metrics:")
print(f"  P(Loss):            {mc_result.probability_of_loss:>10.2%}")
print(f"  CVaR (5%):          {mc_result.expected_shortfall_5:>10.2%}")
print(f"  MaxDD (5th pct):    {mc_result.max_drawdown_5:>10.2%}")
print(f"  MaxDD (median):     {mc_result.max_drawdown_50:>10.2%}")

print(f"\nSharpe Confidence:")
print(f"  Mean:               {mc_result.mean_sharpe:>10.3f}")
print(f"  90% CI:             [{mc_result.ci_5_sharpe:.3f}, {mc_result.ci_95_sharpe:.3f}]")


MONTE CARLO SIMULATION (1000 runs)

Return Distribution:
  Mean:                  -19.66%
  Median:                -31.39%
  Std Dev:                46.44%
  5th percentile:        -66.43%
  95th percentile:        65.08%

Risk Metrics:
  P(Loss):                76.10%
  CVaR (5%):             -72.34%
  MaxDD (5th pct):        23.47%
  MaxDD (median):         47.71%

Sharpe Confidence:
  Mean:                   -2.034
  90% CI:             [-8.820, 3.360]


## 8. Key Insights

**Strategy Comparison**:
- Momentum works in bull markets
- Mean reversion works in ranging/bear
- Multi-factor more robust across conditions
- Long-short provides downside protection

**Risk Analysis**:
- Check P(Loss) <20% for production
- CVaR shows tail risk
- Monte Carlo CI validates Sharpe stability

## Next Steps

- **05_parameter_optimization.ipynb**: Optimize best strategy
- **06_portfolio_optimization.ipynb**: Optimize portfolio construction
- **07_risk_analysis.ipynb**: Deep risk analysis